## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-25-22-54-55 +0000,nypost,Tim Walz compares Trump’s ICE crackdown in Min...,https://nypost.com/2026/01/25/us-news/tim-walz...
1,2026-01-25-22-48-27 +0000,nyt,Most Fox News Reporting on Minneapolis Shootin...,https://www.nytimes.com/2026/01/25/business/me...
2,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
3,2026-01-25-22-45-40 +0000,cbc,Liberals welcome 'spirit of co-operation' from...,https://www.cbc.ca/news/politics/liberals-cons...
4,2026-01-25-22-45-08 +0000,nyt,Deputy attorney general takes a measured tone ...,https://www.nytimes.com/live/2026/01/25/us/min...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2447/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,shooting,29
3,trump,26
16,minneapolis,26
119,alex,16
120,pretti,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2026-01-25-22-29-51 +0000,nypost,LA’s top fed prosecutor Bill Essayli throws fu...,https://nypost.com/2026/01/25/us-news/las-bill...,120
103,2026-01-25-15-31-54 +0000,bbc,Video shows moments around fatal shooting of A...,https://www.bbc.com/news/videos/cp372pqq2rlo?a...,106
89,2026-01-25-16-46-09 +0000,nyt,"In Court Filings, Witnesses Describe Fatal Min...",https://www.nytimes.com/2026/01/24/us/witnesse...,104
139,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,103
167,2026-01-25-04-19-46 +0000,bbc,What we know about fatal shooting of Alex Pret...,https://www.bbc.com/news/articles/c20zjyxep99o...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,120,2026-01-25-22-29-51 +0000,nypost,LA’s top fed prosecutor Bill Essayli throws fu...,https://nypost.com/2026/01/25/us-news/las-bill...
100,56,2026-01-25-15-41-03 +0000,cbc,"No plans for China free trade deal, Carney say...",https://www.cbc.ca/news/politics/carney-canada...
5,41,2026-01-25-22-42-49 +0000,wapo,Live updates: Border Patrol sidesteps question...,https://www.washingtonpost.com/nation/2026/01/...
110,37,2026-01-25-14-55-26 +0000,cbc,"Massive winter storm across U.S. brings snow, ...",https://www.cbc.ca/news/world/us-storm-thousan...
2,33,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
159,30,2026-01-25-06-02-10 +0000,nypost,Florida Rep. Maxwell Frost says he was punched...,https://nypost.com/2026/01/25/us-news/florida-...
33,28,2026-01-25-21-27-13 +0000,nypost,Bay Area ex-mayor candidate staged wife’s kill...,https://nypost.com/2026/01/25/us-news/bay-area...
35,27,2026-01-25-21-25-09 +0000,startribune,"Department of Justice renews push for voter, w...",https://www.startribune.com/department-of-just...
88,27,2026-01-25-17-00-00 +0000,wsj,U.S. Speaker of the House Mike Johnson has hel...,https://www.wsj.com/politics/republicans-house...
83,25,2026-01-25-17-55-08 +0000,nyt,New York City Schools Announce Remote Learning...,https://www.nytimes.com/2026/01/25/us/nyc-scho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
